In [ ]:
#https://www.kaggle.com/code/ludovicocuoghi/twitter-sentiment-analysis-with-bert-roberta/notebook#Tweets-Deep-Cleaning

In [1]:
!pip install simpletransformers
!pip install emoji

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-05-19 08:25:00.079821: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14544514093014956995
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11320098816
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16835913529010599095
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
 xla_global_id: 416903419]

In [3]:
# import some necessary sub libraries
import pandas as pd
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import os
import tarfile
import re
#data processing
import re, string
import emoji
import nltk

#transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

In [4]:
def dict_to_df(d):
    df=pd.DataFrame(d.items())
    df.set_index(0, inplace=True)
    return df

def pack_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

def unpack_model(model_name=''):
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

def prepare_results(result_dict):
  result_dict["f1_macro"] = result_dict["f1"]["f1_macro"]
  result_dict["f1_micro"] = result_dict["f1"]["f1_micro"]
  result_dict["f1_weighted"] = result_dict["f1"]["f1_weighted"]

  result_dict["precision_macro"] = result_dict["precision"]["precision_macro"]
  result_dict["precision_micro"] = result_dict["precision"]["precision_micro"]
  result_dict["precision_weighted"] = result_dict["precision"]["precision_weighted"]

  result_dict["recall_macro"] = result_dict["recall"]["recall_macro"]
  result_dict["recall_micro"] = result_dict["recall"]["recall_micro"]
  result_dict["recall_weighted"] = result_dict["recall"]["recall_weighted"]

  del result_dict["recall"]
  del result_dict["precision"]
  del result_dict["f1"]

  p_per_class = result_dict["prfs"][0]
  r_per_class = result_dict["prfs"][1]
  f_per_class = result_dict["prfs"][2]
  sample_per_class = result_dict["prfs"][3]

  i = 0
  for number in p_per_class:
    result_dict["precision_class_" + str(i)] = number
    i = i + 1
  
  i = 0
  for number in r_per_class:
    result_dict["recall_class_" + str(i)] = number
    i = i + 1
  
  i = 0
  for number in f_per_class:
    result_dict["f1_score_class_" + str(i)] = number
    i = i + 1
  
  i = 0
  for number in sample_per_class:
    result_dict["sample_class_" + str(i)] = number
    i = i + 1
  
  del result_dict["prfs"]

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, precision_recall_fscore_support, balanced_accuracy_score

def f1_multiclass(labels, preds):
  f1_score_info = {
      "f1_micro": f1_score(labels, preds, average='micro'),
      "f1_macro": f1_score(labels, preds, average='macro'),
      "f1_weighted": f1_score(labels, preds, average='weighted')
  }
  return f1_score_info

def p_multiclass(labels, preds):
  precision_info = {
      "precision_micro": precision_score(labels, preds, average='micro'),
      "precision_macro": precision_score(labels, preds, average='macro'),
      "precision_weighted": precision_score(labels, preds, average='weighted')
  }
  return precision_info

def recall_multiclass(labels, preds):
  recall_info = {
      "recall_micro": recall_score(labels, preds, average='micro'),
      "recall_macro": recall_score(labels, preds, average='macro'),
      "recall_weighted": recall_score(labels, preds, average='weighted')
  }
  return recall_info

In [5]:
# ai4bharat/indic-bert
model_name = "bert-base-multilingual-cased"
model_type = "bert"
label_count = 3

In [6]:
#data = pd.read_excel("dataset_urdu.xlsx")
#data.tail()

train = pd.read_csv("Corona_NLP_train.csv",encoding='ISO-8859-1')
test = pd.read_csv("Corona_NLP_test.csv")
#val = pd.read_csv("tweets-valid.csv")
train = train[['OriginalTweet','Sentiment']]
test = test[['OriginalTweet','Sentiment']]

#train["label"] = train["label"].apply(lambda x: 2 if x == -1 else (1 if x == 1 else 0))
#test["label"] = test["label"].apply(lambda x: 2 if x == -1 else (1 if x == 1 else 0))
#val["label"] = val["label"].apply(lambda x: 2 if x == -1 else (1 if x == 1 else 0))
train.head()
train.Sentiment.value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [7]:
##CUSTOM DEFINED FUNCTIONS TO CLEAN THE TWEETS

#Clean emojis from text
def strip_emoji(text):
    return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji

#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

In [8]:
texts_new = []
for t in train.OriginalTweet:
    texts_new.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(t))))))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  """


In [9]:
texts_new_test = []
for t in test.OriginalTweet:
    texts_new_test.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(t))))))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  """


In [10]:
train['text_clean'] = texts_new
test['text_clean'] = texts_new_test

In [11]:
train['text_clean'].head()

0                                              and and
1    advice talk to your neighbours family to excha...
2    coronavirus australia woolworths to give elder...
3    my food stock is not the only one which is emp...
4    me ready to go at supermarket during the covid...
Name: text_clean, dtype: object

In [12]:
text_len = []
for text in train.text_clean:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

In [13]:
train['text_len'] = text_len

In [14]:
text_len_test = []
for text in test.text_clean:
    tweet_len = len(text.split())
    text_len_test.append(tweet_len)

test['text_len'] = text_len_test

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns

#plt.figure(figsize=(7,5))
#ax = sns.countplot(x='text_len', data=train[train['text_len']<10], palette='mako')
#plt.title('Training tweets with less than 10 words')
#plt.yticks([])
#ax.bar_label(ax.containers[0])
#plt.ylabel('count')
#plt.xlabel('')
#plt.show()

In [16]:
train = train[train['text_len'] > 4]
test = test[test['text_len'] > 4]

In [18]:
import numpy as np

In [19]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

token_lens = []

for txt in train['text_clean'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    
max_len=np.max(token_lens)

In [20]:
print(f"MAX TOKENIZED SENTENCE LENGTH: {max_len}")

MAX TOKENIZED SENTENCE LENGTH: 100


In [21]:
token_lens = []

for i,txt in enumerate(train['text_clean'].values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    if len(tokens)>80:
        print(f"INDEX: {i}, TEXT: {txt}") 

INDEX: 1622, TEXT: zsah policie proti spekulantm s roukami na mj popud hejtman steckho kraje ve spoluprci s podle krizovho zkona zajistil tm 700 tisrouek od firmy kter je mla dodat na zdravotnkm ale na posledn chvli se snaila navyovat cenu spolutozvladneme
INDEX: 13623, TEXT: hoy aplaudo a mi segunda familia aplaudoanuestrosheroes aquellos con los que he compartido tantas noches de trabajo y tanta alegra s que como siempre dan todo por el bien de su comunidad presidente por ellos tambin cuarentenanacionalya cuidemosalosquecuidan
INDEX: 16548, TEXT: bir mddettir spermarketlerin lojistik hizmeti avusturya ordusu desteiyle yaplyor dn corona tedavisi iin 22 milyon luk bir aratrma gelitirme btesi aklad hkmet geen hafta da 35 milyon luk 2 yardm paketi aklanmt viyanadan haberler bu kadar
INDEX: 36953, TEXT: 11 nisan cumartesi itibariyle bbnin tm hizmetleri sokaa kma serbestisi olanlar iin devam edecek halk ekmek ve hamidiye su 100 retime geti bb tm stanbulun gda ihtiyacna yetecek kapasitededi

In [22]:
train['token_lens'] = token_lens

In [23]:
train = train.sort_values(by='token_lens', ascending=False)

In [24]:
train = train.iloc[12:]

In [25]:
train = train.sample(frac=1).reset_index(drop=True)

In [26]:
token_lens_test = []

for txt in test['text_clean'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens_test.append(len(tokens))
    
max_len=np.max(token_lens_test)

In [27]:
token_lens_test = []

for i,txt in enumerate(test['text_clean'].values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens_test.append(len(tokens))
    if len(tokens)>80:
        print(f"INDEX: {i}, TEXT: {txt}")

INDEX: 286, TEXT: so hard to decide as much as i want to hodl my 2 ccdcv4 token our place is declared to lock down due to covid19 i will use this to buy some food to stock txnid093bd1db0c0d3a62af15883138a5f57d4cef35ae14e31e602b74489dd2524c7f my b
INDEX: 345, TEXT: informoval jsem zstupce vech obchodnch etzc o aktulnch opatench vldy etzce jsou zsobovny na 95 take nen dvod panikait zsoby potravin fakt nedojdou nen opravdu dvod dnes obsadit a vykoupit supermarkety
INDEX: 2380, TEXT: ahora seguid llorando por el papel higinico que no he comprado porque an tengo seguid creando histeria y preocupacin poniendo fotos de gente en pnico y estanteras vacas que yo seguir yendo a comercios responsables de barrio donde nos cuidan hoy y siempre gracias


In [28]:
test['token_lens'] = token_lens_test

In [29]:
test = test.sort_values(by='token_lens', ascending=False)
test = test.iloc[5:]

In [30]:
test = test.sample(frac=1).reset_index(drop=True)

In [31]:
train['Sentiment'] = train['Sentiment'].map({'Extremely Negative':2,'Negative':2,'Neutral':0,'Positive':1,'Extremely Positive':1})

test['Sentiment'] = test['Sentiment'].map({'Extremely Negative':2,'Negative':2,'Neutral':0,'Positive':1,'Extremely Positive':1})

In [32]:
train['Sentiment'].value_counts()

1    17999
2    15364
0     7560
Name: Sentiment, dtype: int64

In [33]:
import numpy as np
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(train['text_clean']).reshape(-1, 1), np.array(train['Sentiment']).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['text_clean', 'Sentiment']);

In [35]:
train_os['Sentiment'].value_counts()

1    17999
2    17999
0    17999
Name: Sentiment, dtype: int64

In [34]:
train_os.head()

,text_clean,Sentiment
0,sas says restoring balance to the consumer goo...,1
1,full circle research reveals increased consume...,2
2,to the woman who tried to cut the line outside...,2
3,todays heartbreaking covid19 business weakness...,1
4,lysol laundry sanitizer additive crisp linen s...,0


In [36]:
MAX_LEN=128

def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [37]:
X_train = train["text_clean"].values
X_test = test["text_clean"].values

In [39]:
y_train = train["Sentiment"].values
y_test = test["Sentiment"].values

In [38]:
train_input_ids, train_attention_masks = tokenize(X_train, MAX_LEN)
test_input_ids, test_attention_masks = tokenize(X_test, MAX_LEN)

In [41]:
# nun zum Modell
# define hyperparameter
train_args ={"reprocess_input_data": True,
             "learning_rate": 1e-5,
             #"fp16":False,
             "num_train_epochs": 4,
             "overwrite_output_dir":True,
             "train_batch_size": 32,
             "eval_batch_size": 32,
             "gradient_accumulation_steps": 16,
             "max_seq_length": 128
             }

# Create a ClassificationModel
# Attention: Adjust labels accordingly

model = ClassificationModel(
    model_type, model_name,
    num_labels=label_count,
    args=train_args
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [45]:
train_os.head()

,text_clean,Sentiment
0,sas says restoring balance to the consumer goo...,1
1,full circle research reveals increased consume...,2
2,to the woman who tried to cut the line outside...,2
3,todays heartbreaking covid19 business weakness...,1
4,lysol laundry sanitizer additive crisp linen s...,0


In [48]:
train_versuch = pd.DataFrame(X_train)
train_versuch["Sentiment"] = y_train

In [49]:
train_versuch.head()

,0,Sentiment
0,sas says restoring balance to the consumer goo...,1
1,full circle research reveals increased consume...,2
2,to the woman who tried to cut the line outside...,2
3,todays heartbreaking covid19 business weakness...,1
4,lysol laundry sanitizer additive crisp linen s...,0


In [ ]:
#model.train_model(train_os)
model.train_model(train_versuch)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/40923 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/1279 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/1279 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/1279 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/1279 [00:00<?, ?it/s]

In [43]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# CHANGE HERE TEST DATA WITH VAL DATA?
result, model_outputs, wrong_predictions = model.eval_model(test, acc=accuracy_score, recall=recall_multiclass, precision=p_multiclass, bal_acc=balanced_accuracy_score, f1=f1_multiclass, prfs=precision_recall_fscore_support)

In [26]:
# davor iwas mit 52
prepare_results(result)
result_df = dict_to_df(result)
result_df = result_df.sort_values(0)
result_df

,1
0,
acc,0.699631
bal_acc,0.721089
eval_loss,0.746038
f1_macro,0.681379
f1_micro,0.699631
f1_score_class_0,0.591017
f1_score_class_1,0.689417
f1_score_class_2,0.763703
f1_weighted,0.705429


In [ ]:
# PREDICT ON TEST DATA
to_predict = test.tweet.tolist()
preds, outputs = model.predict(to_predict)

sub_df = pd.DataFrame(outputs, columns=[0,1,2])


sub_df.index = test.index
sub_df = sub_df[[0,1,2]]
sub_df.head()
#sub_df.to_csv('outputs/submission.csv', index=False)

In [ ]:
test.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
result_df.to_csv("/content/drive/MyDrive/Projektmodul/Result_Dfs/mara_albert_v2_baseline.csv", encoding="utf-8")

In [ ]:
pack_model('/content/outputs','/content/drive/MyDrive/Projektmodul/mara_albert_v2_baseline_model')